# Different ways of modelling straight line fitting

This notebook is a simple record of the different ways that a straight line fitting can be carried out with varying degrees of complexity. We start with pre-made functions from numpy and statsmodels, building up to fully bayesian covariance models.

In [25]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from itertools import product
import pymc4 as pm
import arviz as az
from statsmodels.regression.linear_model import OLS

In [ ]:
# Create fictional straight line
n_val = 70
x = np.arange(0, n_val, 1)
y = 3 * x + 8 * 1 + stats.norm.rvs(0, 3, n_val)

#%%
b = np.ones(len(x))
X = np.zeros((n_val, 2))

X[:, 0] = x
X[:, 1] = b

# Numpy polyfit

In [34]:
# polyfit automatically adds bias term so only include first column
print(np.polyfit(X[:,0], y, deg = 1))

[2.97736315 8.60819055]


# Statsmodels OLS

In [31]:
# Statsmodels will provide more information of the fit, given p-values, CI etc
model = OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 3.868e+04
Date:                Wed, 30 Sep 2020   Prob (F-statistic):           1.95e-95
Time:                        15:32:49   Log-Likelihood:                -164.09
No. Observations:                  70   AIC:                             332.2
Df Residuals:                      68   BIC:                             336.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             2.9774      0.015    196.667      0.000       2.947       3.008
const          8.6082      0.605     14.222      0.000       7.400       9.816
==============================================================================
Omnibus:                        1.664   Durbin-Watson:                   1.708
Prob(Omnibus):                  0.435   Jarque-Bera (JB):                1.219
Skew:                           0.026   Prob(JB):                        0.544
Kurtosis:                       2.356   Cond. No.                         79.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Closed Form

In [36]:
weights = np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, y))
print(weights)

[2.97736315 8.60819055]


# Gradient Descent

In [21]:
def gradient_descent(X, y, theta, alpha, n_val, n_iter):
    
    y = y.reshape((n_val, 1))
    for _ in np.arange(n_iter):
        y_hat = X.dot(theta)
        error = X.T.dot(y_hat - y) / n_val
        theta_new = theta - alpha * X.T.dot(y_hat - y) / n_val
        theta = theta_new
    return theta_new


def gradient_descent_SGD(X, y, theta, alpha, n_val, n_iter):
    
    y = y.reshape((n_val, 1))
    for _ in np.arange(n_iter):
        for j in np.arange(n_val):
            rand_int = np.random.randint(0, n_val)
            X_i = X[rand_int, :].reshape(1, 2)
            y_i = y[rand_int, :].reshape(1, 1)
            y_hat = X_i.dot(theta)
            error = X_i.T.dot(y_hat - y_i) / n_val
            theta_new = theta - alpha * X_i.T.dot(y_hat - y_i) / n_val
            theta = theta_new
    return theta

def gradient_descent_minibatch(X, y, theta, alpha, n_val, n_iter):
    
    y = y.reshape((n_val, 1))
    for _ in np.arange(n_iter):
        indices = np.random.permutation(n_val)
        X = X[indices, :]
        y = y[indices, :]
        for j in np.arange(0, n_val, 5):
            X_i = X[j:j+5, :]
            y_i = y[j:j+5, :]
            y_hat = X_i.dot(theta)
            error = X_i.T.dot(y_hat - y_i) / n_val
            theta_new = theta - alpha * X_i.T.dot(y_hat - y_i) / n_val
            theta = theta_new
    return theta

#%%
theta = np.array([[1], [1]]) 
theta_gd = gradient_descent(X, y, theta, 0.001, n_val, 200000)
theta = np.array([[1], [1]])
theta_sgd = gradient_descent_SGD(X, y, theta, 0.001, n_val, 20000)
theta = np.array([[1], [1]])
theta_minibatch = gradient_descent_minibatch(X, y, theta, 0.001, n_val, 20000)

In [50]:
print(theta_gd)
print(theta_sgd)
print(theta_minibatch)

[[3.0017324 ]
 [7.74266205]]
[[3.01208627]
 [7.70217484]]
[[2.99983257]
 [7.70388884]]


In [46]:
#%%

# bayesian approach - grid search

slope_grid = np.arange(1, 5, 0.1)
intercept_grid = np.arange(5, 10, 0.1)
total_grid = np.mgrid[1.:5.:0.1, 5.:10.:0.1].reshape(2, -1).T

results = []
for params in total_grid:
    slope_prior = stats.norm.pdf(params[0], loc = 5, scale = 2)
    intercept_prior = stats.norm.pdf(params[1], loc = 7, scale = 2)
    results.append(np.sum(stats.norm.pdf(y, loc = params[0] * X[:, 0] + params[1] * X[:, 1], scale = 4)) * \
              slope_prior * intercept_prior)
        
results /= np.sum(results)

In [47]:
total_grid[results.argmax()]

array([3. , 7.1])

In [8]:
# bayesian approach - HMC w/ pymc4

@pm.model
def linear_fit(data, obs):
    slope_prior = yield pm.Normal(name = 'slope', loc = 4, scale = 1)
    intercept_prior = yield pm.Normal(name = 'intercept', loc = 6, scale = 2)
    scale_prior = yield pm.Uniform(name = 'std', low = 0, high = 10)
    likelihood = yield pm.Normal(name = 'data', loc = data[:,0] * slope_prior + data[:, 1] * intercept_prior, scale = scale_prior, observed = obs)
    return likelihood

estimation = linear_fit(X, y)
trace = pm.sample(estimation, num_samples = 1000)

print(az.summary(trace))

                       mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
linear_fit/slope      3.006  0.017   2.973    3.039      0.000    0.000   
linear_fit/intercept  7.529  0.690   6.220    8.793      0.014    0.010   
linear_fit/std        3.043  0.262   2.584    3.547      0.004    0.003   

                      ess_mean  ess_sd  ess_bulk  ess_tail  r_hat  
linear_fit/slope        3013.0  3012.0    3015.0    3909.0   1.00  
linear_fit/intercept    2313.0  2313.0    2311.0    2408.0   1.01  
linear_fit/std          4414.0  4070.0    5006.0    4259.0   1.00  


In [ ]:
# bayesian approach w/ covariance- HMC w/ pymc4

@pm.model
def linear_fit(data, obs):
    slope_prior = yield pm.Normal(name = 'slope', loc = 4, scale = 1)
    intercept_prior = yield pm.Normal(name = 'intercept', loc = 6, scale = 2)
    scale_prior = yield pm.Uniform(name = 'std', low = 0, high = 10)
    likelihood = yield pm.Normal(name = 'data', loc = data[:,0] * slope_prior + data[:, 1] * intercept_prior, scale = scale_prior, observed = obs)
    return likelihood

estimation = linear_fit(X, y)
trace = pm.sample(estimation, num_samples = 1000)

print(az.summary(trace))

# Plotting Results

In [ ]:
#%%

#plot results - matplotlib
plt.plot(x, y)

#plot results - seaborn
import seaborn as sns
sns.relplot(x = x, y = y, kind = 'scatter')

#plot results - plotly
from plotly.offline import plot
import plotly.graph_objects as go

trace = go.Scatter(x = x, y = y)
fig = go.Figure(data = [trace])
plot(fig)